### Numpy 方式实现卷积层

In [1]:
import numpy as np
"""
TensorFlow有两种数据格式NHWC和NCHW，默认的数据格式是NHWC，可以通过参数data_format指定数据格式。
这个参数规定了 input Tensor 和 output Tensor 的排列方式。
设置为 “NHWC” 时，排列顺序为 [batch, height, width, channels]
设置为 “NCHW” 时，排列顺序为 [batch, channels, height, width]
"""

'\nTensorFlow有两种数据格式NHWC和NCHW，默认的数据格式是NHWC，可以通过参数data_format指定数据格式。\n这个参数规定了 input Tensor 和 output Tensor 的排列方式。\n设置为 “NHWC” 时，排列顺序为 [batch, height, width, channels]\n设置为 “NCHW” 时，排列顺序为 [batch, channels, height, width]\n'

In [10]:
def conv_numpy(x, w, b, pad,strides):
    out = None
    
    N, H, W, C = x.shape
    F,  HH, WW, C = w.shape
  
    X = np.pad(x, ((0, 0),(pad, pad), (pad, pad), (0, 0) ), 'constant')

    Hn = 1 + int((H + 2 * pad - HH) / strides[0])
    Wn = 1 + int((W + 2 * pad - WW) / strides[1])
    
    out = np.zeros((N, Hn, Wn, F))
    
    for n in range(N):
        for m in range(F):
            for i in range(Hn):
                for j in range(Wn):
                    data = X[n, i * strides[0]:i * strides[0] + HH, j * strides[1]:j * strides[1] + WW, :].reshape(1, -1)
                    filt = w[m].reshape(-1, 1)
                    out[n, i, j, m] = data.dot(filt) + b[m]

    return out

In [11]:
inputs_ = np.random.random((10,28,28,3))

In [12]:
w = np.random.random((6,3,3,3))
b = np.random.random((6,))

In [13]:
inputs_ = inputs_.astype(np.float32)
w = w.astype(np.float32)
b = b.astype(np.float32)

In [14]:
a= conv_numpy(inputs_,w = w, b=b, pad=1,strides=(1,1) )

In [15]:
a.shape


(10, 28, 28, 6)

In [36]:
tf.keras.layers.Conv2D(filters=6,kernel_size=(3,3))(inputs_).shape


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



TensorShape([10, 26, 26, 6])

### tf版本实现卷积层

In [10]:
import tensorflow as tf

In [50]:
def corr2d(x, w, b, pad, stride):
    N, H, W, C = tf.shape(x)
    F, HH, WW, C = tf.shape(w)

    x = tf.pad(x, ((0, 0),(pad, pad), (pad, pad), (0, 0) ), 'constant')
    Hn = 1 + int((H + 2 * pad - HH) / stride[0])
    Wn = 1 + int((W + 2 * pad - WW) / stride[1])
    Y = tf.Variable(tf.zeros((N, Hn, Wn, F),dtype=tf.float32))

    for m in range(F):
        for i in range(Hn):
            for j in range(Wn):
                data = x[:, i * stride[0]:i * 1 + HH, j * stride[1]:j * 1 + WW, :]
                filt = w[m,:,:,:]
                Y[:, i, j, m].assign(tf.reduce_sum(tf.multiply(data,filt),axis=(1,2,3))+b[m])

    return Y

In [31]:
x = tf.constant(inputs_)
w_tf = tf.constant(w)
b_tf = tf.constant(b)

In [32]:
x = tf.dtypes.cast(x,dtype=tf.float32)
b_tf = tf.dtypes.cast(w_tf,dtype=tf.float32)
b_tf = tf.dtypes.cast(b_tf,dtype=tf.float32)

In [ ]:
corr2d(x,w,b,1,(1,1))

In [ ]:
class Conv2D(tf.keras.layers.Layer):
    def __init__(self,filters=None,kernel_size=(3,3),stride=(1,1),pad = 0, **kwargs):
        self.filters = filters
        self.kernel_size = kernel_size
        self.stride = stride
        self.pad = 0 
        
        super(Conv2D,self).__init__(**kwargs)


    def build(self, input_shape):
        self.w = self.add_weight(name='w',
                                shape=(self.filters,input_shape[-1],self.kernel_size[0],self.kernel_size[1]),
                                initializer=tf.random_normal_initializer())
        self.b = self.add_weight(name='b',
                                shape=(self.filters,),
                                initializer=tf.random_normal_initializer())
        
        super(Conv2D,self).build(input_shape)
        
    def call(self, inputs):
        return self.corr2d(inputs, self.w, self.b,self.pad,self.stride)
    

    @staticmethod
    def corr2d(x, w, b, pad, stride):
        N, H, W, C = tf.shape(x)
        F, HH, WW, C = tf.shape(w)
        
        Hn = 1 + int((H + pad - HH) / stride[0])
        Wn = 1 + int((W + pad - WW) / stride[1])
        Y = tf.Variable(tf.zeros((N, Hn, Wn, F),dtype=tf.float32))

        for m in range(F):
            for i in range(Hn):
                for j in range(Wn):
                    data = x[:, i * 1:i * 1 + HH, j * 1:j * 1 + WW, :]
                    filt = w[m,:,:,:]
                    Y[:, i, j, m].assign(tf.reduce_sum(tf.multiply(data,filt),axis=(1,2,3))+b[m])

        return Y

In [28]:
conv2 = Conv2D(filters=6,kernel_size=(3,3),stride=(1,1),pad = 0)

In [ ]:
conv2(x)

In [34]:
x = tf.random.uniform((10,3,3,3))
w = tf.random.uniform((6,3,3,3))


In [35]:
tf.reduce_sum(tf.multiply(x,w),axis=(1,2,3))

<tf.Tensor: id=388, shape=(10,), dtype=float32, numpy=
array([5.9358387, 4.895552 , 5.4961605, 5.9548607, 4.543584 , 5.6806326,
       5.212574 , 5.0141573, 5.491521 , 5.8198013], dtype=float32)>

In [8]:
help(tf.multiply)

Help on function multiply in module tensorflow.python.ops.math_ops:

multiply(x, y, name=None)
    Returns x * y element-wise.
    
    *NOTE*: `tf.multiply` supports broadcasting. More about broadcasting
    [here](http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html)
    
    Args:
      x: A `Tensor`. Must be one of the following types: `bfloat16`, `half`, `float32`, `float64`, `uint8`, `int8`, `uint16`, `int16`, `int32`, `int64`, `complex64`, `complex128`.
      y: A `Tensor`. Must have the same type as `x`.
      name: A name for the operation (optional).
    
    Returns:
      A `Tensor`. Has the same type as `x`.



In [ ]:
x = tf.random.uniform((10,28,28,3))
w = tf.random.uniform((6,3,3,3))
b = tf.random.uniform((6,))
x = tf.dtypes.cast(x,tf.float32)
N, H, W, C = tf.shape(x)
F, HH, WW,C = tf.shape(w)


Hn = 1 + int((H + 2 * 0 - HH) / 1)
Wn = 1 + int((W + 2 * 0 - WW) / 1)
Y = tf.Variable(tf.zeros((N, Hn, Wn, F)))

for n in range(N):
    for m in range(F):
        for i in range(Hn):
            for j in range(Wn):
                data = x[n, i * 1:i * 1 + HH, j * 1:j * 1 + WW, :]
                
                print(data.shape)
                filt = w[n]
                print(tf.matmul(data,filt).shape)
                Y[n, i, j, m].assign(tf.cast(tf.matmul(data,filt) ,dtype=tf.float32))

In [70]:
a = Conv2D()

In [75]:
tf

In [ ]:
corr2d(x,w,b)

In [32]:
X = tf.constant([[0,1,2], [3,4,5], [6,7,8]])
K = tf.constant([[0,1], [2,3]])
a(X)

<tf.Tensor: id=864, shape=(1, 1), dtype=float32, numpy=array([[0.61994416]], dtype=float32)>

# 池化层

In [1]:

import tensorflow as tf
import numpy as np

In [9]:
def max_pool_forward_naive(x, pool_size=(2,2),strides=(1,1)):

    N, H, W, C = x.shape
    h_p, w_p = pool_size
    h_s, w_s = strides
    
    
    Hn = 1 + int((H - h_p) / h_s)
    Wn = 1 + int((W - w_p) / w_s)
    out = np.zeros((N, Hn, Wn, C))
    for i in range(Hn):
        for j in range(Wn):
            out[:, i, j, :] = np.max(x[:, i*h_s:i*h_s+h_p, j*w_s:j*w_s+w_p, :], axis=(1,2))
    return out

In [21]:
x = np.random.random((10,26,26,6))

In [22]:
numpy_pool_res = max_pool_forward_naive(x,pool_size=(2,2),strides=(1,1))

In [23]:
numpy_pool_res.shape

(10, 25, 25, 6)

In [24]:
x =tf.constant(x,dtype=tf.float32)

In [29]:
def pool2d(X, pool_size=(2,2),strides=(1,1)):
    N, H, W, C = tf.shape(X)
    p_h, p_w = pool_size
    s_h, s_w = strides
    Y = tf.zeros((N, (H - p_h + 1)//s_h, (W - p_w +1)//s_w, C))
    Y = tf.Variable(Y)

    for i in tf.range(tf.shape(Y)[1]):
        for j in tf.range(tf.shape(Y)[2]):
            Y[:,i,j,:].assign(tf.math.reduce_max(X[:,i*s_h:i*s_h+p_h, j*s_w:j*s_w+p_w,:],axis=(1,2),keepdims=False))
    return Y

In [41]:
tf_pool_res = pool2d(x,pool_size=(2,2),strides=(2,2))

In [42]:
tf_pool_res.shape

TensorShape([10, 12, 12, 6])

In [ ]:
tf_pool_res

In [ ]:
numpy_pool_res

In [44]:
max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')

In [ ]:
max_pool_2d(x)